In [2]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
%matplotlib qt

# Finding Chessboard Corners to Calibrate Camera

In [3]:
# Prepare object points, ex: (0,0,0), (1,0,0), (2,0,0), ..., (8,5,0)
objp = np.zeros((9*6,3),np.float32)
objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

# Arrays to store the object points and image points from all the calibration images.
objpts = [] # 3d points in real world space
imgpts = [] # 2d points in image plane

# Make a list of the calibration images
cal_imgs = glob.glob('camera_cal/calibration*.jpg')

# Cycle through the list and perform chessboard corner search
for fname in cal_imgs:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img_size = gray.shape[::-1]
    
    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (9,6), None)
    
    # If found, add object points and image points to arrays
    if ret:
        objpts.append(objp)
        imgpts.append(corners)
        
        # Draw and display the corners on calibration images
        img = cv2.drawChessboardCorners(img, (9,6), corners, ret)
        cv2.imshow('img', img)
        cv2.waitKey(500)

cv2.destroyAllWindows()

# Calibrate camera

In [4]:
# Calibrate camera using the image points generated from previous step
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpts, imgpts, img_size, None, None)

In [15]:
gray = cv2.imread('camera_cal/calibration1.jpg')

In [16]:
dst = cv2.undistort(gray, mtx, dist, None, mtx)

In [17]:
cv2.imshow('test', dst)
cv2.waitKey(500)

-1